In [ ]:
# 📌 1. Introduction
"""
Problem Overview:
- Binary decision variables
- Linear constraints
- Quadratic objective function

Notebook Goals:
- Reformulate the problem for quantum optimization
- Solve using quantum algorithms (e.g., VQE)
- Validate using classical methods
- Compare performance and solution quality
"""

# 📁 2. Import Libraries
import numpy as np
import matplotlib.pyplot as plt
from qiskit import Aer
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit.algorithms import VQE
from qiskit.circuit.library import TwoLocal
from scipy.optimize import minimize

# 📊 3. Define the Binary Optimization Problem
Q = np.array([[1, -1], [-1, 2]])  # Quadratic objective coefficients
A = np.array([[1, 1]])            # Linear constraint coefficients
b = np.array([1])
# You can expand this based on your figure or real data

# 📐 4. Reformulate to Unconstrained Form
lambda_penalty = 10
def penalized_objective(x):
    return x.T @ Q @ x + lambda_penalty * np.sum(np.square(A @ x - b))

# 🎯 5. Quantum Formulation Using VQE
qp = QuadraticProgram()
qp.binary_var('x0')
qp.binary_var('x1')
qp.minimize(quadratic={
    ('x0', 'x0'): Q[0][0],
    ('x0', 'x1'): Q[0][1],
    ('x1', 'x1'): Q[1][1]
})
qp.linear_constraint(linear={'x0': 1, 'x1': 1}, sense='<=' , rhs=1)

ansatz = TwoLocal(2, 'ry', 'cz')
vqe = VQE(ansatz=ansatz, quantum_instance=Aer.get_backend('aer_simulator'))
optimizer = MinimumEigenOptimizer(vqe)
result_q = optimizer.solve(qp)

# 🧪 6. Classical Validation
def classical_cost(x):
    return penalized_objective(np.array(x))

initial_guess = [0.5, 0.5]
res_cl = minimize(classical_cost, initial_guess, bounds=[(0,1)]*2)
x_cl = res_cl.x

# 📈 7. Compare Results
print("🔷 Quantum Solution:", result_q.variables_dict)
print("🔶 Classical Solution:", x_cl)

# Optional: Track convergence, runtime, scalability

# 🎓 8. Conclusion
"""
Discuss:
- Accuracy and feasibility of both solutions
- Strengths and limitations of classical vs quantum
- Future potential of quantum optimization
"""
